In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/mollyagostino/Desktop/Motor_Vehicle_Collisions_-_Crashes.csv')

df['CRASH DATETIME'] = pd.to_datetime(df['CRASH DATE'] + ' ' + df['CRASH TIME'], errors='coerce')

vehicle_cols = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
                'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
df_vehicles = df.melt(id_vars=['BOROUGH'], value_vars=vehicle_cols,
                      var_name='VEHICLE_TYPE_CODE_NUM', value_name='VEHICLE_TYPE')

df_vehicles.dropna(subset=['VEHICLE_TYPE'], inplace=True)

df_vehicles['VEHICLE_TYPE'] = df_vehicles['VEHICLE_TYPE'].str.upper().str.strip()

vehicle_map = {
    'STATION WAGON': 'STATION WAGON/SUV',
    'STATION WAGON/SUV': 'STATION WAGON/SUV',
    'STATION WAGON (SPORT UTILITY VEHICLE)': 'STATION WAGON/SUV',
    'SPORT UTILITY VEHICLE': 'STATION WAGON/SUV',
    'SUV': 'STATION WAGON/SUV',
    'VAN': 'VAN',
    'BOX TRUCK': 'BOX TRUCK',
    'PICK-UP TRUCK': 'PICK-UP TRUCK',
    'TAXI': 'TAXI',
    'CAB': 'TAXI',
    'PASSENGER VEHICLE': 'PASSENGER VEHICLE',  
    'SEDAN': 'PASSENGER VEHICLE', 
    'UNKNOWN': 'UNKNOWN', 
    'SPORT UTILITY/STATIONWAGON': 'STATION WAGON/SUV', 
    'STATION WAGON/SPORT UTILITY VEHICLE': 'STATION WAGON/SUV',
    'SPORT UTILITY VEHICLE (SUV)': 'STATION WAGON/SUV',
}

df_vehicles['VEHICLE_TYPE'] = df_vehicles['VEHICLE_TYPE'].replace(vehicle_map)

print(df_vehicles['VEHICLE_TYPE'].unique())


top_types = df_vehicles['VEHICLE_TYPE'].value_counts().head(6).index
df_vehicles['VEHICLE_TYPE_GROUPED'] = df_vehicles['VEHICLE_TYPE'].apply(lambda x: x if x in top_types else 'OTHER')

filtered = df_vehicles[df_vehicles['VEHICLE_TYPE'].isin(top_types)]

grouped = filtered.groupby(['BOROUGH', 'VEHICLE_TYPE']).size().reset_index(name='COUNT')

pivot = grouped.pivot(index='BOROUGH', columns='VEHICLE_TYPE', values='COUNT').fillna(0)

plt.figure(figsize=(12, 8))  # Adjusted figure size
pivot.plot(kind='bar', stacked=False, figsize=(12, 8))  # Adjusted size again
plt.title('Most Common Vehicle Types in Collisions by Borough')
plt.ylabel('Crash Count')
plt.xlabel('Borough')
plt.xticks(rotation=45, ha='right')  # Rotate and align x-axis labels
plt.legend(title='Vehicle Type', bbox_to_anchor=(1.05, 1))  # Moved legend outside
plt.tight_layout()  # Ensures everything fits
plt.show()
